In [6]:
# imports
import os
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

import gensim

import nltk
#file for punkt splitter
nltk.download('punkt');
#file for vader sentiment
nltk.download('vader_lexicon');

#wordnet lemmatization
nltk.download('wordnet')
#more for preprocessing
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()

import pyLDAvis.gensim

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=20,20
%matplotlib inline

import warnings; warnings.simplefilter('ignore')


C:\Users\emiel\.virtualenvs\Thesis_Notebook-tvt70hiV\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<br>

## 1: Load the dataset <a class="anchor" id="chapter1"></a>

In [2]:
dataset_path_true = os.path.join("sources", "ISOT", "True.csv")
dataset_path_fake = os.path.join("sources", "ISOT", "Fake.csv")

df_true = pd.read_csv(dataset_path_true, encoding='utf-8') # make sure to use the right encoding
df_fake = pd.read_csv(dataset_path_fake, encoding='utf-8') 

dfm_true = df_true.head()
dfm_fake = df_fake.head()

display(dfm_true)
display(dfm_fake)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


<br>

## 2: Split the text into sentences <a class="anchor" id="chapter2"></a>

in lab 03 we used a big manual function, but for now we will use the nltk tokenizer here :)

In [3]:
def split_sentences(article_text):
    """Takes a string, returns a list of its individual sentences ()"""
    return pd.Series(nltk.tokenize.sent_tokenize(article_text))

sample_sentences = split_sentences(dfm_true.text[0])
display(sample_sentences.iloc[[0,1,-2,-1]])

0     WASHINGTON (Reuters) - The head of a conservat...
1     In keeping with a sharp pivot under way among ...
28    The package far exceeded the $44 billion reque...
29             The Senate has not yet voted on the aid.
dtype: object

### 2.1 Corpus of Sentences

In [4]:
corpus_sentences = [] # a list of all documents (by sentence)

amount of documents in corpus:  44898


### 2.2 Corpus of Entire Texts

In [ ]:
corpus_texts = [] # list of all documents (by entire body)

#corpus_texts = df_true['text'].tolist() + df_fake['text'].tolist() 
corpus_texts = df_true['text'].tolist() + df_fake['text'].tolist() 

print("amount of documents in corpus: ", len(corpus_texts))

<br>

## 3 Text Preprocessing
In order to do proper topic analysis the text needs to become understandable by removing unineteresting properties of the text. We lower case it, stem and lemmatize it, and remove all words under 3 characters or stopwords (it them ...).

In [5]:
def stem_lemmatize(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    output = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            output.append(stem_lemmatize(token))
    return output

print(preprocess(corpus_texts[7]))

['follow', 'statement', 'post', 'verifi', 'twitter', 'account', 'presid', 'donald', 'trump', 'realdonaldtrump', 'potu', 'opinion', 'express', 'reuter', 'edit', 'statement', 'confirm', 'accuraci', 'realdonaldtrump', 'fake', 'news', 'love', 'talk', 'call', 'approv', 'rat', 'foxandfriend', 'show', 'rat', 'approxim', 'presid', 'obama', 'despit', 'massiv', 'neg', 'trump', 'coverag', 'russia', 'hoax', 'unit', 'state', 'post', 'offic', 'lose', 'billion', 'dollar', 'year', 'charg', 'amazon', 'littl', 'deliv', 'packag', 'make', 'amazon', 'richer', 'post', 'offic', 'dumber', 'poorer', 'charg', 'sourc', 'link', 'jpexyr']


<br>

## 4 Dictionary Creation

word embeddings, all occuring words are stored and get a number (embedding) those embeddings can later be used for vector calculations. of course not all words are important, so words that appear more than 100000 times or that exist in >60%  our corpus (those are very likely words slipped the stopword list) and less than 15 time (words not important enought for a topic)

In [5]:
%%time

# Creating the gensim dictionary of word embeddings

if (0):
    dictionary = gensim.corpora.Dictionary.load(os.path.join("gensim", "dictionary"))


if (0):
    
    processed_corpus_texts = [preprocess(d) for d in corpus_texts]

    dictionary = gensim.corpora.Dictionary(processed_corpus_texts)
    print("unfiltered: ", dictionary)

    dictionary.filter_extremes(no_below=15, no_above=0.6, keep_n=100000)
    
print("  filtered: ", dictionary)

C:\Users\emiel\.virtualenvs\Thesis_Notebook-tvt70hiV\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'corpus_texts' is not defined

<br>

## 5 Bag of Words creation
Now we create a vector representation in the form of a bag of words for eacht document. a vector that lets us know how often each word in the preprocessed text, that also exists in our dictionary, occurs

In [10]:
if (0):
    with open('bag_of_words.pickle', 'rb') as f: dictionary = pickle.load(f)

if (0):
    bow_corpus_texts = [dictionary.doc2bow(text) for text in processed_corpus_texts]
    

<br>

## 6 LDA model creation


In [7]:
%%time
if (0):
    lda_model = gensim.models.LdaMulticore.load(os.path.join("gensim", "lda_model"))
if (0):
    lda_model5 = gensim.models.LdaMulticore(bow_corpus_texts, num_topics=5, id2word=dictionary, passes=2, workers=2)
    # multicore speeds up the process significantly

Wall time: 0 ns


<br>

## 7 TF-IDF model creation 


In [ ]:
# train the model
tfidf_model = gensim.models.TfidfModel(bow_corpus_texts)

<br>

## 8 Case Examination
take a look at some classifications

In [25]:
#TODO LDAVIS For TWEAKING SHIT SAVE PICTURES AND STEPS WHOOO

pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(lda_model50, bow_corpus_texts, dictionary)
visualisation

C:\Users\emiel\.virtualenvs\Thesis_Notebook-tvt70hiV\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
35     0.137849  0.094757       1        1  11.406036
22     0.111474 -0.029388       2        1   5.597344
46     0.029598  0.178872       3        1   4.061968
47     0.117293  0.025866       4        1   3.921628
41     0.070828  0.098893       5        1   3.893013
6     -0.000040 -0.033739       6        1   3.668566
20    -0.104688 -0.075604       7        1   3.482417
34     0.127245  0.056749       8        1   3.095002
21     0.086946 -0.134874       9        1   2.976875
42     0.075162 -0.022062      10        1   2.649403
25    -0.014321  0.122990      11        1   2.418987
3      0.065516 -0.015985      12        1   2.350200
4      0.091859 -0.047362      13        1   2.307049
40     0.051525 -0.065008      14        1   2.303356
10    -0.134358  0.051657      15        1   2.275119
37    -0.026814  0.020004      16        1   2.268313
31    -0.134238 -0.043677      17        1   2.214256
12     0.049343  0.062296      18        1   2.099801
39    -0.083858 -0.061382      19        1   2.095175
14    -0.020104  0.001658      20        1   1.851553
24     0.011719  0.153621      21        1   1.831278
36    -0.192952  0.040449      22        1   1.787696
16    -0.084159  0.047334      23        1   1.713697
17     0.045674  0.093041      24        1   1.680007
30     0.061726  0.085275      25        1   1.623086
11    -0.000188 -0.023098      26        1   1.597672
33    -0.049043  0.014036      27        1   1.503883
38    -0.039022 -0.120953      28        1   1.415807
5     -0.157562  0.007724      29        1   1.360697
19     0.101224 -0.048952      30        1   1.353521
44     0.125388 -0.016977      31        1   1.340613
0     -0.083838 -0.104257      32        1   1.321216
45    -0.047498 -0.049564      33        1   1.238387
7      0.023445 -0.119142      34        1   1.198947
23    -0.108534  0.106376      35        1   1.157742
13    -0.089038 -0.018915      36        1   1.136672
1      0.010495  0.040847      37        1   1.047389
29     0.076318 -0.042972      38        1   0.903211
15    -0.023308  0.006846      39        1   0.899856
28    -0.093185  0.008460      40        1   0.823205
9      0.047695 -0.051433      41        1   0.786639
18    -0.061691  0.114680      42        1   0.758268
49     0.030815  0.019233      43        1   0.754018
27    -0.187156 -0.030937      44        1   0.649551
48    -0.020836 -0.076448      45        1   0.644781
8      0.053961  0.019011      46        1   0.632280
2      0.068502 -0.033759      47        1   0.568912
26     0.068538 -0.068948      48        1   0.566744
32    -0.030232 -0.013268      49        1   0.479545
43     0.046527 -0.121971      50        1   0.288621, topic_info=           Term           Freq          Total Category  logprob  loglift
313       obama   33992.000000   33992.000000  Default  30.0000  30.0000
210       trump  111503.000000  111503.000000  Default  29.0000  29.0000
381     clinton   26886.000000   26886.000000  Default  28.0000  28.0000
187       senat   24245.000000   24245.000000  Default  27.0000  27.0000
168  republican   36284.000000   36284.000000  Default  26.0000  26.0000
..          ...            ...            ...      ...      ...      ...
175      reuter     146.921757   18550.765692  Topic50  -5.1213   1.0094
210       trump     172.832633  111503.760243  Topic50  -4.9589  -0.6217
225        year     138.137359   40073.842828  Topic50  -5.1830   0.1776
199       state     130.585318   62355.122867  Topic50  -5.2392  -0.3208
427       polit     118.407911   21335.902478  Topic50  -5.3371   0.6538

[3653 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
8348      13  0.936077        aarp
11360     15  0.967609       ababa
7525       7  0.002917       abadi
7525      22  0.002917       abadi
7525      23  0.005835   

In [8]:
article = 8

print("most frequently occuring word", max(preprocess(corpus_texts[article]),key=preprocess(corpus_texts[article]).count))

for index, score in sorted(lda_model50[bow_corpus_texts[article]],
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model50.print_topic(index, 10)))
    
print("\n\n", preprocess(corpus_texts[article]), "\n\n", corpus_texts[article])

NameError: name 'preprocess' is not defined

In [ ]:
article = 8

print(max(preprocess(corpus_texts[article]),key=preprocess(corpus_texts[article]).count))

for index, score in sorted(tfidf_model[bow_corpus_texts[article]],
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, tfidf_model.print_topic(index, 10)))
    
print("\n\n", preprocess(corpus_texts[article]), "\n\n", corpus_texts[article])

<br>

## X Saving items
because we dont want to sit waiting every time please

In [1]:
if (0):
    #lda_modelXX.save(os.path.join("gensim", "lda_modelXX"))
    
    bow_df = pd.read_csv((os.path.join("gensim", "bag_of_words")))
    bow_corpus_texts = bow_df.to_list()

    
#     dictionary.save(os.path.join("gensim", "dictionary"))

with open(os.path.join("gensim", "bag_of_words.pickle"), 'wb') as f: pickle.dump(bow_corpus_texts, f)

NameError: name 'os' is not defined

<br>

### X: Performing sentiment analysis <a class="anchor" id="chapter3"></a>


In [ ]:
def get_scores(text: list, method='VADER'):
    if method == 'VADER':
        scores = text.apply(lambda s: sia.polarity_scores(s)['compound']) #list of compound score per sentence
    else:
        scores = None

    return scores

sample_scores = get_scores(sample_sentences)
print(sample_scores.iloc[[0,1,-2,-1]])